In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Sample data: user ratings for movies
data = {
    'User': ['User1', 'User1', 'User1', 'User2', 'User2', 'User3', 'User3', 'User3', 'User4', 'User4'],
    'Movie': ['Movie1', 'Movie2', 'Movie3', 'Movie1', 'Movie4', 'Movie2', 'Movie3', 'Movie4', 'Movie1', 'Movie3'],
    'Rating': [5, 3, 4, 4, 2, 5, 4, 1, 4, 5]
}

# Create DataFrame
df = pd.DataFrame(data)

def add_user_ratings(user, ratings):
    """
    Add the new user's ratings to the DataFrame.
    """
    global df
    new_data = {'User': [user]*len(ratings), 'Movie': list(ratings.keys()), 'Rating': list(ratings.values())}
    new_df = pd.DataFrame(new_data)
    df = pd.concat([df, new_df], ignore_index=True)

def recommend_movies(user, num_recommendations=2):
    # Pivot table to create user-item matrix
    user_movie_matrix = df.pivot_table(index='User', columns='Movie', values='Rating').fillna(0)

    # Calculate cosine similarity between users
    user_similarity = cosine_similarity(user_movie_matrix)

    # Create a DataFrame for user similarity
    user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

    if user not in user_similarity_df.index:
        return "User not found."

    # Get the similarity scores for the specified user
    user_similarities = user_similarity_df[user]

    # Get the movies rated by the specified user
    user_ratings = user_movie_matrix.loc[user]

    # Calculate weighted sum of ratings for each movie
    weighted_sum = np.dot(user_similarity_df[user].values, user_movie_matrix.fillna(0).values)

    # Normalize by the sum of similarity scores
    sum_of_similarities = user_similarity_df[user].sum()
    weighted_average_ratings = weighted_sum / sum_of_similarities

    # Create a DataFrame for weighted average ratings
    weighted_ratings_df = pd.DataFrame(weighted_average_ratings, index=user_movie_matrix.columns, columns=['Weighted_Rating'])

    # Filter out movies already rated by the user
    weighted_ratings_df = weighted_ratings_df.loc[~weighted_ratings_df.index.isin(user_ratings[user_ratings > 0].index)]

    # Sort by weighted rating and get top N recommendations
    recommendations = weighted_ratings_df.sort_values(by='Weighted_Rating', ascending=False).head(num_recommendations)

    return recommendations.index.tolist()

def get_user_input():
    user = input("Enter your user ID: ")
    print("Please enter your ratings for the following movies (if you haven't watched a movie, just press Enter):")

    movies = ['Movie1', 'Movie2', 'Movie3', 'Movie4']
    ratings = {}

    for movie in movies:
        rating = input(f"Rating for {movie} (0-5): ")
        if rating:
            ratings[movie] = float(rating)

    return user, ratings

def chat():
    print("Welcome to the movie recommendation system!")
    user, ratings = get_user_input()
    add_user_ratings(user, ratings)

    recommended_movies = recommend_movies(user)
    print(f"Recommended movies for {user}: {recommended_movies}")

# Run the chat function
chat()

Welcome to the movie recommendation system!
Enter your user ID: User5
Please enter your ratings for the following movies (if you haven't watched a movie, just press Enter):
Rating for Movie1 (0-5): 5
Rating for Movie2 (0-5): 
Rating for Movie3 (0-5): 2
Rating for Movie4 (0-5): 4
Recommended movies for User5: ['Movie2']
